# Skew-Confidence-Interval

An algorithm to estimate CI for distribution with high skew.

Generate a distribution $X~\log{(N(\mu, \sigma))}+b$ with unique $\mu, \sigma, b$ match given $m, v, s$ (mean, variance and skewness).

### ps:

Only work when $s > 0$

Use normal when $s := 0$

Input $-s, -m$ when $s < 0$ and consider $-X$ finally.

In [3]:
import numpy as np
import scipy.stats
import scipy

In [26]:
def magic_cubic(x, b):
    # an cubic function to solve parameter of log normal from skewness
    return (x-1)*((x+2)**2) - b

def magic_cubic_deri(x):
    # derivative of cubic function above
    return 3*x*(x+2)

class LogNormal():
    def __init__(self, mean, var, skew, newton_threshold=0.000001):
        if skew <= 0:
            raise ValueError("skewness error, expect positive value, got {}".format(skew))
        if var <= 0:
            raise ValueError("variance error, expect positive value, got {}".format(var))
        
        pow_sigma2 = 1 + skew/9
        for t in range(100):
            if magic_cubic(pow_sigma2, skew**2) < newton_threshold:
                break
            
            pow_sigma2 -= magic_cubic(pow_sigma2, skew**2)/magic_cubic_deri(pow_sigma2)
        
        pow_mu = (var/(pow_sigma2*(pow_sigma2 - 1)))**0.5
        self.bias = mean - pow_mu*(pow_sigma2**0.5)
        self.mu = np.log(pow_mu)
        self.sigma = np.sqrt(np.log(pow_sigma2))
    
    def ppf(self, percent):
        # percent point function
        if (percent > 1) or (percent < 0):
            raise ValueError("percent error, expect 0~1, got {}".format(percent))
        
        return np.exp(self.mu + self.sigma*scipy.stats.norm.ppf(percent))+self.bias

## Test

Consider a condition, throw an unfair coin n times

In [76]:
n = 100
p = 0.1

hits = np.arange(n+1) # 0,1,...,n

combine = (np.arange(n+1) < 2).astype(np.float64) # [1,1,0,0,...,0]
for t in range(n-1):
    combine[1:] += combine[:-1]

prob = combine * np.power(p, hits) * np.power(1-p, n - hits)

mean = (hits * prob).sum()
var = (((hits - mean)**2)*prob).sum()
skew = (((hits - mean)**3)*prob).sum()/(var**1.5)

log_normal = LogNormal(mean, var, skew)

for SL in [0.3, 0.2, 0.1, 0.05, 0.01, 0.001, 0.0001, 0.00001]: #Significance Level
    CI = log_normal.ppf(1-SL)
    print(int(CI), (prob*(hits>CI)).sum()/SL)

11 0.9898896657041567
12 0.9908944367978395
13 1.2387679259933058
15 0.7978105422216691
17 1.0007279262125086
20 0.8075738743662686
23 0.39637423837858254
25 0.40998837116850945
